Perhaps mark each pdf with a code denoting the bank type, so this file only has to do a simple string parse to know what to do. i.e. 'JohnDoeCAP.pdf', 'MarkAnthonyNED.pdf', etc. 

Sometimes Description is long and overflows. I cut it off, but should we keep it all?

In [1]:
import math 
import tabula as tb
from tabula import read_pdf

#constant conversion factor, per adobe
#pdf measurer pdf unit = 1/72 in
cf =72  

#all measurements in format [top,left,bottom,width]

#page counter
from PyPDF2 import PdfFileReader
def count_pdf_pages(file):
    with open(file, 'rb') as pdf:
        page_count = PdfFileReader(pdf).getNumPages()
    return page_count

In [2]:
import pandas as pd
#CAPITEC
#working for the first page

#capitec measurements
#first page:
page_1_box = [3.77, 0.4, 9.8, 8.5,]
#middle pages:
middle_pages_box = [0.3, 0.4, 10.2, 8.5]

file = "Capitec.pdf"
page_count = count_pdf_pages(file)

#convert boxes
for i in range(0, len(page_1_box)):
    page_1_box[i] *= cf
    
for i in range(0, len(middle_pages_box)):
    middle_pages_box[i] *= cf

#get first page
page_1_table = tb.read_pdf(file, pages=1,area=[page_1_box],output_format="dataframe", stream=True)
page_1_df = page_1_table[0].drop(labels='Posting Date', axis=1)

#clean first page
page_1_df = page_1_df.rename(columns={'Transaction': 'Transaction Date'})
page_1_df = page_1_df.drop(index = 0)
page_1_df = page_1_df.dropna(subset=['Transaction Date','Balance (R)'])

#append first page
total_df = pd.DataFrame(columns=page_1_df.columns)
total_df = pd.concat([total_df, page_1_df])


for page in range(2, page_count+1):

    #get middle page
    middle_pages_table = tb.read_pdf(file, pages=page, area=[middle_pages_box],
                                 output_format="dataframe", stream=True, pandas_options={'header': None})
    middle_df = middle_pages_table[0]

    #clean middle page
    #print(middle_df.columns)
    middle_df = middle_df.dropna(axis='columns', how='all')
    middle_df.columns = page_1_df.columns


    middle_df = middle_df.dropna(subset=['Transaction Date','Balance (R)'])
    middle_df['Transaction Date'] = middle_df['Transaction Date'].map(lambda x: str(x).split()[-1])

    #append to total df
    total_df = pd.concat([total_df, middle_df])





total_df


,Transaction Date,Description,Money In (R),Money Out (R),Balance (R)
1,30/08/2021,Branch Card Replacement Fee,NaN,-70.0,-57.64
2,30/08/2021,Cash Deposit Cash Dep Dnr Boksbur Dnr Boksbur...,100.0,NaN,42.36
4,30/08/2021,Cash Deposit Fee (Notes),NaN,-1.3,41.06
5,30/08/2021,Banking App Prepaid Purchase MTN,NaN,-15.0,26.06
6,30/08/2021,Prepaid Mobile Purchase Fee,NaN,-0.5,25.56
...,...,...,...,...,...
9,04/10/2021,ATM Cash Withdrawal Sparkatm Bp Roodekop,NaN,-100.0,338.9
11,04/10/2021,ATM Cash Withdrawal Fee,nk,-7.5,331.4
12,04/10/2021,Payment Received: Rtc 05dtnty7sh Tk Capitec B2...,a,NaN,531.4
13,04/10/2021,Banking App Transfer to Rainy Days (1709362276...,NaN,-195.0,336.4


In [3]:
import pandas as pd

df1 = pd.DataFrame([['a', 1,2,3,4], ['b', 2],['a', 1], ['b', 2],['a', 1]],
                   columns=range(0,5))
df1

,0,1,2,3,4
0,a,1,2.0,3.0,4.0
1,b,2,NaN,NaN,NaN
2,a,1,NaN,NaN,NaN
3,b,2,NaN,NaN,NaN
4,a,1,NaN,NaN,NaN
